# Querying & Retrieving

# 3.1 — Query & Retrieve (no LLM)

Goal: Given a query, encode it, search the FAISS index, and inspect the top hits
and their context. Useful for verifying that retrieval quality looks right before
adding generation.

### Step 1 — Load filenames, texts, encoder, and index

Reads `docs_list.txt` for filenames, loads texts from `RAW`, prepares the
`all-MiniLM-L6-v2` encoder, and opens the persisted FAISS `INDEX`.

In [3]:
%pip install -q sentence-transformers faiss-cpu
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.paths import RAW, DOCS_LIST, INDEX
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

Note: you may need to restart the kernel to use updated packages.


### Step 2 — Encode a query and inspect top-k results

Encodes the query with normalized embeddings, searches FAISS, prints the top hit
filenames, and previews the retrieved context. Adjust `k` as needed.

In [4]:
def rag_respond(q: str) -> str:
    qv = encoder.encode([q], normalize_embeddings=True).astype("float32")
    k  = min(3, index.ntotal)           # don’t ask for more than we have
    D, I = index.search(qv, k)

    valid = [i for i in I[0] if 0 <= i < len(docs_text)]
    if not valid:
        return "No results in index or artifact/corpus mismatch. Rebuild 1.1 then 2.1."

    context = "\n\n".join(docs_text[i] for i in valid)
    # ... OpenAI call ...
    sources = ", ".join(dict.fromkeys(names[i] for i in valid))
    return f"{answer}\n\n---\nSources: {sources}"
